In [7]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import datetime
import warnings
warnings.filterwarnings("ignore")

In [8]:
version = "v1.0"
data_url = "../data/flights.csv"

In [9]:
import sys  
#sys.path.insert(0, 'C:/Users/debgn/MLOPS/backend/src/')
sys.path.insert(0, '../backend/src/')

import data_preprocessing_training
from data_preprocessing_training import preprocess_and_split

In [10]:
#read the data
df = pd.read_csv(data_url)

In [11]:
#shape of the data
df.shape

(5819079, 31)

In [ ]:
#cleaning and preprocessing
X_train,X_test,y_train,y_test = preprocess_and_split(df)

In [ ]:
X_train.head()

,MONTH,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,SCHEDULED_TIME,SCHEDULED_ARRIVAL,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
5272295,11,496,450,-6.0,80.0,1300,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5678727,12,522,391,8.0,144.0,2059,9.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2102837,5,575,393,-1.0,74.0,1113,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5765214,12,501,489,-2.0,125.0,1625,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
950249,3,345,373,-10.0,143.0,1111,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train.columns


NameError: name 'X_train' is not defined

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4655263 entries, 5272295 to 1692743
Data columns (total 27 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   MONTH                  int64  
 1   ORIGIN_AIRPORT         int32  
 2   DESTINATION_AIRPORT    int32  
 3   DEPARTURE_DELAY        float64
 4   SCHEDULED_TIME         float64
 5   SCHEDULED_ARRIVAL      int64  
 6   AIR_SYSTEM_DELAY       float64
 7   SECURITY_DELAY         float64
 8   AIRLINE_DELAY          float64
 9   LATE_AIRCRAFT_DELAY    float64
 10  WEATHER_DELAY          float64
 11  DEPARTURE_TIME_HOUR    int32  
 12  DEPARTURE_TIME_MINUTE  int32  
 13  AIRLINE_AA             float64
 14  AIRLINE_AS             float64
 15  AIRLINE_B6             float64
 16  AIRLINE_DL             float64
 17  AIRLINE_EV             float64
 18  AIRLINE_F9             float64
 19  AIRLINE_HA             float64
 20  AIRLINE_MQ             float64
 21  AIRLINE_NK             float64
 22  AIRLINE_OO       

# 2. Machine Learning Modeling :


Etablison la **Baseline**

Le principe de la baseline en machine learning consiste à définir une référence simple ou une méthode triviale pour prédire les résultats. Cette référence sert à évaluer les performances d’un modèle de manière relative. Si un modèle ne fait pas mieux que cette baseline, il est considéré comme inefficace ou inutile, car il n'apporte pas de valeur ajoutée par rapport à une approche simple.

In [ ]:
import numpy as np
# Average arrival delay for our dataset
baseline_preds = y_train.agg('sum') / y_train.agg('count') 

# Baseline error by average arrival delay 
baseline_errors = abs(baseline_preds - y_test)
print('Average baseline error: ', round(np.mean(baseline_errors),2))

Average baseline error:  21.03


Il s’agit de notre erreur de base moyenne de 21.04 minutes de retard que nous souhaitons surmonter avec notre modèle de régression.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.svm import SVR


# Définition correcte des modèles
Linear_Regression = LinearRegression()
model_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
Decision_Tree = DecisionTreeRegressor(random_state=42)


In [ ]:
# Entraînement des modèles
Linear_Regression.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
Decision_Tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [ ]:
# Prédictions sur les données de test
y_pred_lr = Linear_Regression.predict(X_test)
y_pred_xgb = model_xgb.predict(X_test)
y_pred_dt = Decision_Tree.predict(X_test)

### 1. Mean Squared Error (MSE) - Erreur quadratique moyenne
Le **MSE** mesure la moyenne des carrés des erreurs, c'est-à-dire la différence au carré entre la valeur réelle et la valeur prédite. Plus cette valeur est faible, meilleure est la performance du modèle. Cette valeur reste relative d'un cas ou à un autre 

<img src="../notebooks/images/mse.jpg" alt="Description de l'image" width="500" height="100" />


--------------------------------------------------------------------------------------------------------------------------

### 2. Root Mean Squared Error (RMSE) - Erreur quadratique moyenne racine
Le **RMSE** est simplement la racine carrée du MSE. Il est utilisé pour ramener l'échelle des erreurs à l'échelle originale des données, ce qui le rend plus facile à interpréter.

<img src="../notebooks/images/rmse.png" alt="Description de l'image" width="400" height="200" />


--------------------------------------------------------------------------------------------------------------------------

### 3. Mean Absolute Error (MAE) - Erreur absolue moyenne
Le **MAE** mesure la moyenne des erreurs absolues, c'est-à-dire la différence absolue entre la valeur réelle et la valeur prédite.

<img src="../notebooks/images/mae.jpg" alt="Description de l'image" width="500" height="200" />


--------------------------------------------------------------------------------------------------------------------------

### R² Score (Coefficient de détermination)
Le R² score, également appelé coefficient de détermination, indique la proportion de la variance dans la variable cible qui est expliquée par le modèle. Il est calculé comme suit :

<img src="../notebooks/images/r2.png" alt="Description de l'image" width="500" height="200" />

* Il montre dans quelle mesure le modèle capture la variance des données.
* Un R² proche de 1 indique que le modèle explique une grande partie de la variance des données, tandis qu'un R² proche de 0 indique que le modèle est inefficace.
* Le R² est très utile pour comparer des modèles de régression différents.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Fonction pour calculer les métriques
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "R²": r2}

print("******************************Evaluation Linear Regression********************")
score_lr= evaluate_model(y_test,y_pred_lr)
for metric, value in score_lr.items():
    print(f"{metric}: {value:.4f}")

print("******************************Evaluation model_xgb********************")
score_xgb= evaluate_model(y_test,y_pred_xgb)
for metric, value in score_xgb.items():
    print(f"{metric}: {value:.4f}")

print("******************************Evaluation Decision_Tree********************")
score_dt= evaluate_model(y_test,y_pred_dt)
for metric, value in score_dt.items():
    print(f"{metric}: {value:.4f}")


******************************Evaluation Linear Regression********************
MAE: 7.2508
MSE: 87.0478
RMSE: 9.3299
R²: 0.9431
******************************Evaluation model_xgb********************
MAE: 6.0234
MSE: 79.3021
RMSE: 8.9052
R²: 0.9482
******************************Evaluation Decision_Tree********************
MAE: 6.8937
MSE: 104.6761
RMSE: 10.2311
R²: 0.9316


#### Finetuning du model_xgb

In [ ]:
import xgboost as xgb

# Initialiser le modèle XGBoost
xgb_model = xgb.XGBRegressor(random_state=42)

# Définir l'espace de recherche pour les hyperparamètres
param_grid = {
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [50, 100, 200],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.2]
}


from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,  # Nombre d'itérations aléatoires
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print("Meilleurs hyperparamètres (Random Search) : ", best_params)

# Récupérer le meilleur modèle
best_model = random_search.best_estimator_



Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [ ]:
y_pred = best_model.predict(X_test)

print("******************************Evaluation Best model xgboost********************")
score_best= evaluate_model(y_test,y_pred)
for metric, value in score_best.items():
    print(f"{metric}: {value:.4f}")





******************************Evaluation Best model xgboost********************
MAE: 6.1876
MSE: 85.1925
RMSE: 9.2300
R²: 0.9443


In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Charger les fichiers airlines et airports
airlines = pd.read_csv("../data/airlines.csv")  # Suppose une colonne "AIRLINE" existe
airports = pd.read_csv("../data/airports.csv")  # Suppose une colonne "AIRPORT" existe

# Obtenir les listes d'airlines et d'airports
airline_list = airlines['IATA_CODE'].tolist()
airport_list = airports['IATA_CODE'].tolist()

# Fonction pour générer une ligne de données
def generate_row():
    month = random.randint(1, 12)  # Mois entre 1 et 12
    origin_airport = random.choice(airport_list)
    destination_airport = random.choice(airport_list)
    while destination_airport == origin_airport:
        destination_airport = random.choice(airport_list)
    
    departure_delay = round(random.uniform(-20.0, 120.0), 1)  # Retard au départ en minutes
    scheduled_time = round(random.uniform(30.0, 300.0), 1)  # Durée prévue du vol en minutes
    scheduled_arrival = random.randint(0, 2359)  # Heure prévue d'arrivée (format 24h sans ":")
    air_system_delay = round(random.uniform(0.0, 30.0), 1)  # Retard système
    security_delay = round(random.uniform(0.0, 15.0), 1)  # Retard sécurité
    airline_delay = round(random.uniform(-10.0, 30.0), 1)  # Retard de la compagnie
    late_aircraft_delay = round(random.uniform(0.0, 20.0), 1)  # Retard avion précédent
    weather_delay = round(random.uniform(0.0, 10.0), 1)  # Retard météo
    airline = random.choice(airline_list)
    departure_time = datetime(2024, month, random.randint(1, 28), 
                              random.randint(0, 23), random.randint(0, 59))  # Date et heure départ
    
    return {
        "MONTH": month,
        "ORIGIN_AIRPORT": origin_airport,
        "DESTINATION_AIRPORT": destination_airport,
        "DEPARTURE_DELAY": departure_delay,
        "SCHEDULED_TIME": scheduled_time,
        "SCHEDULED_ARRIVAL": scheduled_arrival,
        "AIR_SYSTEM_DELAY": air_system_delay,
        "SECURITY_DELAY": security_delay,
        "AIRLINE_DELAY": airline_delay,
        "LATE_AIRCRAFT_DELAY": late_aircraft_delay,
        "WEATHER_DELAY": weather_delay,
        "DEPARTURE_TIME": departure_time,
        "AIRLINE": airline
    }

# Générer plusieurs lignes
def generate_data(n=100):
    data = [generate_row() for _ in range(n)]
    return pd.DataFrame(data)

# Génération des données
synthetic_data = generate_data(1)  # Exemple pour 10 lignes
print(synthetic_data)




   MONTH ORIGIN_AIRPORT DESTINATION_AIRPORT  DEPARTURE_DELAY  SCHEDULED_TIME  \
0      9            ISP                 KTN            117.3            92.3   

   SCHEDULED_ARRIVAL  AIR_SYSTEM_DELAY  SECURITY_DELAY  AIRLINE_DELAY  \
0                629               8.3             4.2            5.7   

   LATE_AIRCRAFT_DELAY  WEATHER_DELAY      DEPARTURE_TIME AIRLINE  
0                 14.0            0.4 2024-09-08 07:10:00      VX  


In [ ]:
import sys  
#sys.path.insert(0, 'C:/Users/debgn/MLOPS/backend/src/')
sys.path.insert(0, '../backend/src/')
import preprocessing
from preprocessing import preprocess
a= preprocess(synthetic_data)

FileNotFoundError: [Errno 2] No such file or directory: '../airport_encoder.pkl'

In [ ]:
# Charger l'encodeur sauvegardé
with open('../airport_encoder.pkl', 'rb') as f:
    airport_encoder = pickle.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '../airport_encoder.pkl'